# Text Extraction with Albert

> Albert



Resource: [Text Extraction with Bert](https://keras.io/examples/nlp/text_extraction_with_bert/)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers[sentencepiece]
!pip install tokenizers

     |████████████████████████████████| 2.1MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 27.0MB/s 
     |████████████████████████████████| 901kB 42.1MB/s 
     |████████████████████████████████| 1.1MB 21.9MB/s 


In [ ]:
import json
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import transformers
from transformers import TFAlbertModel, AlbertTokenizerFast, AlbertTokenizer

In [ ]:
def tokens_to_offsets(original_text, tokenized_text):
  offsets = []
  special_tokens_mask = tokenized_text['special_tokens_mask']
  tokens = tokenizer.convert_ids_to_tokens(tokenized_text["input_ids"])
  index = 0
  for i in range(len(tokens)):
    if special_tokens_mask[i] == 1:
      offsets.append((0, 0))
    else:
      new_index = index + len(tokens[i])
      offsets.append((index, new_index))

      while (new_index < len(original_text) and original_text[new_index] == ' '):
        new_index += 1
      index = new_index
  return offsets

In [ ]:
class WikiElement:
    def __init__(self, question, context, answer, answer_start, answer_end):
        self.question = question
        self.context = context
        self.answer = answer
        self.answer_start = answer_start
        self.answer_end = answer_end
        
    def preprocess(self):
        # create context vector with answers marked
        context_vector = [0] * len(self.context)
        for index in range(self.answer_start, self.answer_end):
            context_vector[index] = 1
            
        # tokenize context   
        tokenized_context = tokenizer(self.context, return_offsets_mapping=True)
        context_offsets = tokenized_context['offset_mapping']

        # find answer token indices 
        answer_token_index = []
        for index, (start, end)  in enumerate(context_offsets):
            if sum(context_vector[start:end]) > 0: # if token is answer
                answer_token_index.append(index)
        
        if len(answer_token_index) == 0:
            return 0
        
        # start and end token index
        start_token_index = answer_token_index[0]
        end_token_index = answer_token_index[-1]
        
        # tokenize question
        tokenized_question = tokenizer(self.question, return_special_tokens_mask=True)

        # create inputs       
        input_ids = tokenized_context['input_ids'] + tokenized_question['input_ids'][1:]
        token_type_ids = [0] * len(tokenized_context['input_ids']) + [1] * len(tokenized_question['input_ids'][1:])
            
        attention_mask = [1] * len(input_ids)
        
        # padding for equal lenght sequence
        padding_length = max_len - len(input_ids)
        if padding_length > 0: # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length) # len(input) [1] + padding [0]
            token_type_ids = token_type_ids + ([0] * padding_length) # context [0] + question [1] + padding [0]
        elif padding_length < 0:
            return 0
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_index = start_token_index
        self.end_token_index = end_token_index
        self.context_token_to_char = context_offsets
        return 1
            
      
    def class_print(self):
        print("Question: {}\nAnswer: {}\nAnswer Start: {}\nAnswer End: {}\nContext: {}".format(self.question, 
                                                                                              self.answer,  
                                                                                              self.answer_start, 
                                                                                              self.answer_end,
                                                                                              self.context))

In [ ]:
def read_json(file_name):
    with open(file_name, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)
    return data

def json_to_list(json_dataset):
    dataset = []
    for paragraph_element in json_dataset["data"]:
        for question_element in paragraph_element["qas"]:
            dataset.append(WikiElement(question_element["question"],
                                       paragraph_element["text"],
                                       question_element["answer"],
                                       question_element["answer_start"],
                                       question_element["answer_end"]))
    print("Number of questions: ", len(dataset))
    return dataset

def create_input_targets(dataset):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_index": [],
        "end_token_index": [],
    }
    i=0
    for item in dataset:
        # print(i)
        i = i + 1
        # print(item.class_print())
        for key in dataset_dict:
            dataset_dict[key].append(getattr(item, key))
            
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
        
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    
    y = [dataset_dict["start_token_index"], dataset_dict["end_token_index"]]
    return x, y

def find_max_length(dataset):
    max_ = 0
    index = 0
    i = 0
    for element in dataset:
        tokenized_question = tokenizer(element.question)
        tokenized_context = tokenizer(element.context)
        input_ids = tokenized_context['input_ids'] + tokenized_question['input_ids'][1:]
        
        if len(input_ids) > max_:
            max_ = len(input_ids)
            index = i
        i += 1
        
    print("Max length: {}, Index: {}".format(max_, index))
    return max_

def train_test_split(dataset):
    random.shuffle(dataset) 
    cut = int(len(dataset)*0.1)
    train, test = dataset[:-cut], dataset[-cut:] 
    
    return train, test

def create_model():
    encoder = TFAlbertModel.from_pretrained(MODEL_NAME)
    
    # QA model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(input_ids,
                        token_type_ids=token_type_ids, 
                        attention_mask=attention_mask)[0]
    
    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)
    
    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)
    
    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)
    
    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model


## Load Tokenizer

In [ ]:
path = "/content/gdrive/MyDrive/Q&A projesi/"
models_path = path + "models/"
MODEL_NAME = "loodos/albert-base-turkish-uncased"
model_save_name = MODEL_NAME.split("/")[0] + "-" + MODEL_NAME.split("/")[1] + "/"
save_path = models_path + model_save_name

In [ ]:
tokenizer = AlbertTokenizerFast.from_pretrained(MODEL_NAME, do_lower_case=False, keep_accents=True)

In [ ]:
tokenizer("Hey naber", return_offsets_mapping=True)

{'input_ids': [2, 1419, 98, 390, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (3, 5), (5, 9), (0, 0)]}

In [ ]:
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
tokenizer.save_pretrained(save_path)

('/content/gdrive/MyDrive/Q&A projesi/models/loodos-albert-base-turkish-uncased/tokenizer_config.json',
 '/content/gdrive/MyDrive/Q&A projesi/models/loodos-albert-base-turkish-uncased/special_tokens_map.json',
 '/content/gdrive/MyDrive/Q&A projesi/models/loodos-albert-base-turkish-uncased/spiece.model',
 '/content/gdrive/MyDrive/Q&A projesi/models/loodos-albert-base-turkish-uncased/added_tokens.json')

## Load Dataset

In [ ]:
file_path = path + "json_dataset/wiki_dataV1_combined.json"
json_dataset = read_json(file_path)
json_dataset["data"][0]["qas"][1]

{'answer': 'Bulgaristan',
 'answer_end': 196,
 'answer_start': 185,
 'id': 1,
 'question': "Türkiye'nin kuzeybatısındaki komşusu kimdir?"}

In [ ]:
raw_dataset = json_to_list(json_dataset)
raw_dataset[0].class_print()

Number of questions:  2234
Question: Türkiye'nin topraklarının büyük bölümü nerededir?
Answer: Anadolu
Answer Start: 69
Answer End: 76
Context: Türkiye Cumhuriyeti ya da kısaca Türkiye, topraklarının büyük bölümü Anadolu'da, küçük bir bölümü ise Balkan Yarımadası'nın güneydoğu uzantısı olan Trakya'da yer alan ülke. Kuzeybatıda Bulgaristan, batıda Yunanistan, kuzeydoğuda Gürcistan, doğuda Ermenistan, İran ve Azerbaycan'ın ekslav toprağı Nahçıvan, güneydoğuda ise Irak ve Suriye komşusudur. Güneyini Kıbrıs adası ve Akdeniz. Batısını Ege Denizi ve kuzeyini Karadeniz çevreler. Marmara Denizi ise İstanbul Boğazı ve Çanakkale Boğazı ile birlikte Anadolu'yu Trakya'dan yani Asya'yı Avrupa'dan ayırır. Türkiye, Avrupa ve Asya'nın kavşak noktasında yer alması sayesinde önemli bir jeostratejik güce sahiptir.


In [ ]:
max_len = 512

dataset = []
for data in raw_dataset:
    flag = data.preprocess()
    if flag == 1:
      dataset.append(data)
    
print("Dataset len: ", len(dataset))

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors


Dataset len:  2013


In [ ]:
train, test = train_test_split(dataset)

x_train, y_train = create_input_targets(train)
x_test, y_test = create_input_targets(test)

print(len(x_train[0]), len(y_train[0]), len(x_test[0]), len(y_test[0]))

1812 1812 201 201


In [ ]:
configuration = transformers.AlbertConfig()  # default parameters and configuration for BERT

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.20.97.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.20.97.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
All model checkpoint layers were used when initializing TFAlbertModel.

All the layers of TFAlbertModel were initialized from the model checkpoint at loodos/albert-base-turkish-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_albert_model_1 (TFAlbertMode TFBaseModelOutputWit 11939584    input_4[0][0]                    
                                                                 input_6[0][0]              

In [ ]:
# Load Weights from Drive
# model.load_weights(path + "models/bertV1_weights.h5")

model.fit(
    x_train,
    y_train,
    epochs=5,  # For demonstration, 3 epochs are recommended
    verbose=2,
    batch_size=64,
)

Epoch 1/5
29/29 - 6s - loss: 0.3565 - activation_2_loss: 0.1983 - activation_3_loss: 0.1581
Epoch 2/5
29/29 - 6s - loss: 0.3147 - activation_2_loss: 0.1541 - activation_3_loss: 0.1607
Epoch 3/5
29/29 - 6s - loss: 0.3883 - activation_2_loss: 0.1870 - activation_3_loss: 0.2013
Epoch 4/5
29/29 - 6s - loss: 0.4528 - activation_2_loss: 0.2082 - activation_3_loss: 0.2447
Epoch 5/5
29/29 - 6s - loss: 0.4301 - activation_2_loss: 0.2006 - activation_3_loss: 0.2295


In [ ]:
pred_start, pred_end = model.predict(x_test)
count = 0
before_count = 0
results = []
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
  element = test[idx]
  offsets = element.context_token_to_char
  start = np.argmax(start)
  end = np.argmax(end)

  if start >= len(offsets):
    continue

  pred_char_start = offsets[start][0]

  if end < len(offsets):
    pred_char_end = offsets[end][1]
    pred_ans = element.context[pred_char_start:pred_char_end]
  else:
    pred_ans = element.context[pred_char_start:]
  results.append({
      "question": element.question,
      "true answer": element.answer,
      "predicted answer": pred_ans,
      "context": element.context
  })
  # print(f"Question: {element.question}")
  # print(f"Prediction: {pred_ans}\nTrue Answer: {element.answer}")
  # print(f"Context: {element.context}")
  # print("\n")
  print("'" + pred_ans + "'" + "  ->'" + element.answer + "'")
  if pred_ans == element.answer:
    before_count += 1
  if len(pred_ans) > 0 and pred_ans[0] == ' ':
    pred_ans = pred_ans[1:]
  if pred_ans == element.answer:
    count += 1

acc = count / len(y_test[0])
acc2 = before_count / len(y_test[0])
print(f"accuracy:={acc:.2f}")
print(f"before accuracy:={acc2:.2f}")

' Sklera'  ->'ağ tabaka'
' suyu Turpgiller'  ->'Turpgiller'
' fahri doktor'  ->'fahri doktor'
' 28 Şubat 2017'  ->'13 Nisan 1922'
' aylık abonelik sistemiyle'  ->'aylık abonelik sistemiyle'
' evreni inceleyen daldır. Bu sebeple evreni genel olarak inceler; evreni doğuşundan sonuna kadar inceler, evrenin doğası ve içerdiklerinin doğasına kadar, evrene dair her şeyle ilişkilidir. Bundan farklı olarak, kozmogoni sadece evrenin doğuşu, kökeni ile ilgilidir'  ->'kozmogeni'
''  ->'altın bilet'
' dört'  ->'Finlandiya'
' Charles Gounod'  ->'Charles Gounod'
' II'  ->'çelik kasalı bir tanksavar mayını'
''  ->'Stratum germinativum'
' Suyun kaldırma kuvveti'  ->'Suyun kaldırma kuvveti'
' Eski Mısır'  ->'Eski Mısır'
''  ->'Joseph Black'
' firari askerler ve sivil gönüllülerden'  ->'Çoğunlukla firari askerler ve sivil gönüllüler'
' Yunanistan'  ->'Gürcistan'
' dört ayın adlarını değiştiren yasa ile ayın adı kasım yapıldı. 1946 yılında, Refet Ülgen eylül ayı için ilk güz, kasım ayı içinse songüz adın

In [ ]:
results[0]["question"]

'Başlıca İslam mezhepleri nelerdir?'

In [ ]:
with open(save_path + "test-results/" + model_save_name.split("/")[0] + "_10epoch_result.txt", "w") as f:
  for result in results:
    f.write('%s\n' %result)

### Save Weights to Google Drive

In [ ]:
model.save_weights(save_path + "weights/" + model_save_name.split("/")[0] + "_10epoch_weights.h5")